```bash
pip install tensorflow-gpu==1.2
pip install dm-sonnet -U
pip install tensorflow-probability==0.5.0
```

In [1]:
import numpy as np
import tensorflow as tf
from sklearn.utils import shuffle
import re
import time
import collections
import os
from dnc import DNC

/usr/local/lib/python3.6/dist-packages/tensorflow/python/util/tf_inspect.py:75: DeprecationWarning: inspect.getargspec() is deprecated since Python 3.0, use inspect.signature() or inspect.getfullargspec()
  return _inspect.getargspec(target)


In [2]:
def build_dataset(words, n_words, atleast=1):
    count = [['PAD', 0], ['GO', 1], ['EOS', 2], ['UNK', 3]]
    counter = collections.Counter(words).most_common(n_words)
    counter = [i for i in counter if i[1] >= atleast]
    count.extend(counter)
    dictionary = dict()
    for word, _ in count:
        dictionary[word] = len(dictionary)
    data = list()
    unk_count = 0
    for word in words:
        index = dictionary.get(word, 0)
        if index == 0:
            unk_count += 1
        data.append(index)
    count[0][1] = unk_count
    reversed_dictionary = dict(zip(dictionary.values(), dictionary.keys()))
    return data, count, dictionary, reversed_dictionary

In [3]:
with open('english-train', 'r') as fopen:
    text_from = fopen.read().lower().split('\n')[:-1]
with open('vietnam-train', 'r') as fopen:
    text_to = fopen.read().lower().split('\n')[:-1]
print('len from: %d, len to: %d'%(len(text_from), len(text_to)))

len from: 500, len to: 500


In [4]:
concat_from = ' '.join(text_from).split()
vocabulary_size_from = len(list(set(concat_from)))
data_from, count_from, dictionary_from, rev_dictionary_from = build_dataset(concat_from, vocabulary_size_from)
print('vocab from size: %d'%(vocabulary_size_from))
print('Most common words', count_from[4:10])
print('Sample data', data_from[:10], [rev_dictionary_from[i] for i in data_from[:10]])

vocab from size: 1935
Most common words [(',', 564), ('.', 477), ('the', 368), ('and', 286), ('to', 242), ('of', 220)]
Sample data [482, 483, 78, 6, 137, 484, 10, 226, 787, 14] ['rachel', 'pike', ':', 'the', 'science', 'behind', 'a', 'climate', 'headline', 'in']


In [5]:
concat_to = ' '.join(text_to).split()
vocabulary_size_to = len(list(set(concat_to)))
data_to, count_to, dictionary_to, rev_dictionary_to = build_dataset(concat_to, vocabulary_size_to)
print('vocab to size: %d'%(vocabulary_size_to))
print('Most common words', count_to[4:10])
print('Sample data', data_to[:10], [rev_dictionary_to[i] for i in data_to[:10]])

vocab to size: 1461
Most common words [(',', 472), ('.', 430), ('tôi', 283), ('và', 230), ('có', 199), ('chúng', 196)]
Sample data [84, 22, 668, 73, 10, 389, 110, 34, 81, 299] ['khoa', 'học', 'đằng', 'sau', 'một', 'tiêu', 'đề', 'về', 'khí', 'hậu']


In [6]:
GO = dictionary_from['GO']
PAD = dictionary_from['PAD']
EOS = dictionary_from['EOS']
UNK = dictionary_from['UNK']

In [7]:
for i in range(len(text_to)):
    text_to[i] += ' EOS'

In [8]:
num_reads = 5
num_writes = 1
memory_size = 128
word_size = 128
clip_value = 20

In [9]:
class Chatbot:
    def __init__(self, size_layer, num_layers, embedded_size, 
                 from_dict_size, to_dict_size, learning_rate, batch_size,
                attn_input_feeding=True):
        
        def attn_decoder_input_fn(inputs, attention):
            if attn_input_feeding:
                return inputs
        
        def attention(encoder_out, cell, seq_len, encoder_last_state, reuse=False):
            attention_mechanism = tf.contrib.seq2seq.BahdanauAttention(num_units = size_layer, 
                                                                    memory = encoder_out,
                                                                    memory_sequence_length = seq_len)
            return tf.contrib.seq2seq.AttentionWrapper(
                cell = cell, 
                attention_mechanism = attention_mechanism,
                attention_layer_size = size_layer,
                cell_input_fn=attn_decoder_input_fn,
                initial_cell_state=encoder_last_state,
                alignment_history=False)
        
        self.X = tf.placeholder(tf.int32, [None, None])
        self.Y = tf.placeholder(tf.int32, [None, None])
        self.X_seq_len = tf.count_nonzero(self.X, 1, dtype=tf.int32)
        self.Y_seq_len = tf.count_nonzero(self.Y, 1, dtype=tf.int32)
        access_config = {
            "memory_size": memory_size,
            "word_size": word_size,
            "num_reads": num_reads,
            "num_writes": num_writes,
        }
        controller_config = {
            "hidden_size": size_layer,
        }
        self.dnc_cell = DNC(access_config=access_config, controller_config=controller_config,
                            output_size=size_layer, clip_value=clip_value)
        self.dnc_initial = self.dnc_cell.initial_state
        
        encoder_embeddings = tf.Variable(tf.random_uniform([from_dict_size, embedded_size], -1, 1))
        encoder_embedded = tf.nn.embedding_lookup(encoder_embeddings, self.X)
        
        initial_state = self.dnc_initial(batch_size)
        self.encoder_out, self.encoder_state = tf.nn.dynamic_rnn(
            cell=self.dnc_cell, inputs=encoder_embedded,
            sequence_length=self.X_seq_len, dtype=tf.float32,
            initial_state=initial_state)
        main = tf.strided_slice(self.Y, [0, 0], [batch_size, -1], [1, 1])
        decoder_input = tf.concat([tf.fill([batch_size, 1], GO), main], 1)
        # decoder
        decoder_embeddings = tf.Variable(tf.random_uniform([to_dict_size, embedded_size], -1, 1))
        decoder_cell = attention(self.encoder_out, self.dnc_cell, self.X_seq_len,self.encoder_state)
        dense_layer = tf.layers.Dense(to_dict_size)
        training_helper = tf.contrib.seq2seq.TrainingHelper(
            inputs = tf.nn.embedding_lookup(decoder_embeddings, decoder_input),
            sequence_length = self.Y_seq_len,
            time_major = False)
        training_decoder = tf.contrib.seq2seq.BasicDecoder(
            cell = decoder_cell,
            helper = training_helper,
            initial_state = decoder_cell.zero_state(batch_size=batch_size, dtype=tf.float32),
            output_layer = dense_layer)
        training_decoder_output, _, _ = tf.contrib.seq2seq.dynamic_decode(
            decoder = training_decoder,
            impute_finished = True,
            output_time_major=False,
            maximum_iterations = tf.reduce_max(self.Y_seq_len))
        
        predicting_helper = tf.contrib.seq2seq.GreedyEmbeddingHelper(
                embedding = decoder_embeddings,
                start_tokens = tf.tile(tf.constant([GO], dtype=tf.int32), [batch_size]),
                end_token = EOS)
        predicting_decoder = tf.contrib.seq2seq.BasicDecoder(
                cell = decoder_cell,
                helper = predicting_helper,
                initial_state = decoder_cell.zero_state(batch_size=batch_size, dtype=tf.float32),
                output_layer = dense_layer)
        predicting_decoder_output, _, _ = tf.contrib.seq2seq.dynamic_decode(
                decoder = predicting_decoder,
                impute_finished = True,
                maximum_iterations = 2 * tf.reduce_max(self.X_seq_len))
        self.training_logits = training_decoder_output.rnn_output
        self.predicting_ids = predicting_decoder_output.sample_id
        masks = tf.sequence_mask(self.Y_seq_len, tf.reduce_max(self.Y_seq_len), dtype=tf.float32)
        self.cost = tf.contrib.seq2seq.sequence_loss(logits = self.training_logits,
                                                     targets = self.Y,
                                                     weights = masks)
        self.optimizer = tf.train.AdamOptimizer(learning_rate).minimize(self.cost)
        y_t = tf.argmax(self.training_logits,axis=2)
        y_t = tf.cast(y_t, tf.int32)
        self.prediction = tf.boolean_mask(y_t, masks)
        mask_label = tf.boolean_mask(self.Y, masks)
        correct_pred = tf.equal(self.prediction, mask_label)
        correct_index = tf.cast(correct_pred, tf.float32)
        self.accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

In [10]:
size_layer = 256
num_layers = 2
embedded_size = 128
learning_rate = 0.001
batch_size = 16
epoch = 20

In [11]:
tf.reset_default_graph()
sess = tf.InteractiveSession()
model = Chatbot(size_layer, num_layers, embedded_size, len(dictionary_from), 
                len(dictionary_to), learning_rate,batch_size)
sess.run(tf.global_variables_initializer())

Instructions for updating:
keep_dims is deprecated, use keepdims instead


/usr/local/lib/python3.6/dist-packages/numpy/lib/type_check.py:546: DeprecationWarning: np.asscalar(a) is deprecated since NumPy v1.16, use a.item() instead
  'a.item() instead', DeprecationWarning, stacklevel=1)


In [12]:
def str_idx(corpus, dic):
    X = []
    for i in corpus:
        ints = []
        for k in i.split():
            ints.append(dic.get(k,UNK))
        X.append(ints)
    return X

In [13]:
X = str_idx(text_from, dictionary_from)
Y = str_idx(text_to, dictionary_to)

In [14]:
def pad_sentence_batch(sentence_batch, pad_int):
    padded_seqs = []
    seq_lens = []
    max_sentence_len = max([len(sentence) for sentence in sentence_batch])
    for sentence in sentence_batch:
        padded_seqs.append(sentence + [pad_int] * (max_sentence_len - len(sentence)))
        seq_lens.append(len(sentence))
    return padded_seqs, seq_lens

In [15]:
for i in range(epoch):
    total_loss, total_accuracy = 0, 0
    for k in range(0, (len(text_to) // batch_size) * batch_size, batch_size):
        index = k+batch_size
        batch_x, seq_x = pad_sentence_batch(X[k: index], PAD)
        batch_y, seq_y = pad_sentence_batch(Y[k: index], PAD)
        predicted, accuracy,loss, _ = sess.run([model.predicting_ids, 
                                                model.accuracy, model.cost, model.optimizer], 
                                      feed_dict={model.X:batch_x,
                                                model.Y:batch_y})
        total_loss += loss
        total_accuracy += accuracy
    total_loss /= (len(text_to) // batch_size)
    total_accuracy /= (len(text_to) // batch_size)
    print('epoch: %d, avg loss: %f, avg accuracy: %f'%(i+1, total_loss, total_accuracy))

epoch: 1, avg loss: 6.409067, avg accuracy: 0.069264
epoch: 2, avg loss: 5.859403, avg accuracy: 0.089123
epoch: 3, avg loss: 5.727355, avg accuracy: 0.105535
epoch: 4, avg loss: 5.574304, avg accuracy: 0.115628
epoch: 5, avg loss: 5.404255, avg accuracy: 0.125040
epoch: 6, avg loss: 5.201616, avg accuracy: 0.142058
epoch: 7, avg loss: 4.965463, avg accuracy: 0.156692
epoch: 8, avg loss: 4.678100, avg accuracy: 0.177471
epoch: 9, avg loss: 4.390165, avg accuracy: 0.191455
epoch: 10, avg loss: 4.087937, avg accuracy: 0.218103
epoch: 11, avg loss: 3.786239, avg accuracy: 0.255813
epoch: 12, avg loss: 3.450217, avg accuracy: 0.298548
epoch: 13, avg loss: 3.129698, avg accuracy: 0.346350
epoch: 14, avg loss: 2.765721, avg accuracy: 0.403250
epoch: 15, avg loss: 2.383927, avg accuracy: 0.466340
epoch: 16, avg loss: 2.102273, avg accuracy: 0.514097
epoch: 17, avg loss: 1.850025, avg accuracy: 0.560891
epoch: 18, avg loss: 1.609209, avg accuracy: 0.605419
epoch: 19, avg loss: 1.363634, avg ac

In [16]:
for i in range(len(batch_x)):
    print('row %d'%(i+1))
    print('QUESTION:',' '.join([rev_dictionary_from[n] for n in batch_x[i] if n not in [0,1,2,3]]))
    print('REAL ANSWER:',' '.join([rev_dictionary_to[n] for n in batch_y[i] if n not in[0,1,2,3]]))
    print('PREDICTED ANSWER:',' '.join([rev_dictionary_to[n] for n in predicted[i] if n not in[0,1,2,3]]),'\n')

row 1
QUESTION: and as you can imagine , i hated that moment of ripping with incredible intensity .
REAL ANSWER: vì tôi bị bỏng 70 % cơ thể nên mất khoảng 1 tiếng tháo băng .
PREDICTED ANSWER: vì tôi huýt gió cùng với cơn đau . 

row 2
QUESTION: and i would try to reason with them and say , &quot; why don &apos;t we try something else ?
REAL ANSWER: như bạn có thể tưởng tượng tôi căm ghét cái khoảnh khắc bóc toạc với 1 sức mạnh kinh hồn .
PREDICTED ANSWER: và 1 : 1 : bóc rất thích , &quot; &quot; do tốt khắc bộ phim không chưa bao giờ thường quên . 

row 3
QUESTION: why don &apos;t we take it a little longer -- maybe two hours instead of an hour -- and have less of this intensity ? &quot;
REAL ANSWER: và tôi sẽ cố gắng lý sự với họ &quot; tại sao chúng ta không thử cách khác ? &quot;
PREDICTED ANSWER: và người này vào , không phải đến mọi người không đi trẻ ? 

row 4
QUESTION: and the nurses told me two things .
REAL ANSWER: &quot; tại sao chúng ta không làm lâu hơn 1 chút 2 tiếng thay